**ПРИМЕЧАНИЕ.**

Целью проекта не было побить SOTA результаты.
Также целью я не ставил сделать этот мир лучше, создав, например, перводчик с русского на суахили.
Несколько месяцев назад мне на глаза попалась статья о том, как использовали GPT
дообучив модель на тексте переписки умершей девушки. Статья запала в душу.
И мне захотелось, что называется "потрогать" эту технологию.
Но из-за нехватки времени не доходили руки.
Потом я записался на этот курс по NLP и решил в рамках курса попробовать самому
дообучить модель. Так как использование чужой перписки для публичного проекта
неэтично, то мной было принято решение использовать стихи известного поэта.
Я начал с Пушкина, но потом решил сменить его на Маяковского, так как у последнего более узнаваемый стиль.
К моему сожалению использовать в colab большую и среднюю модель не получилось, 
пришлось ограничиться маленькой((.
В конце этого файла идут ссылки на некоторые статьи, которые мне пришлось читать в рамках этого проекта. 


P.S. К перписке с другим человеком я еще вернусь, но сделаю это для личного пользования, непублично.

Вот ссылка на ту статью.
https://habr.com/ru/news/t/576952/

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#!git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np

np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
#torch.cuda.empty_cache()

cuda


In [ ]:
# Воспользуемся русскоязычной версией GPT от Сбера.
# Large и medium к сожалению, бесплатный colab не тянул,
# поэтому придётся пользоваться версией small (((. 

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"

#model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
#model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/479aa59074c4dcd4c36106252da033d03bc92e3010947ce1d3714de224c2af1f.7362c0dbb32f750eeea5a5b93bbd0c6876eac41453369265d5a49df1c9142b6f
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/sberbank

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling


# Первоначально я дообучал модель на стихах Пушкина.
# Но потом решил переключиться на Маяковского, так как у него более узнаваемый стиль.

train_path = 'drive/MyDrive/Colab Notebooks/ods/majakovskiy.txt'

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
#block_size: It refers to the windows size that is moved across the text file. Set to -1 to use maximum allowed length.

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file drive/MyDrive/Colab Notebooks/ods/cached_lm_GPT2Tokenizer_64_majakovskiy.txt [took 0.001 s]


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    
    output_dir="drive/MyDrive/Colab Notebooks/ods/output",
    overwrite_output_dir=True, #overwrite the content of the output directory
    
    #logging_strategy="epoch",
    num_train_epochs=150, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
    gradient_accumulation_steps=16, # to make "virtual" batch size larger Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#torch.cuda.empty_cache()
trainer.train()

***** Running training *****
  Num examples = 24
  Num Epochs = 150
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 150


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=150, training_loss=0.09098412831624349, metrics={'train_runtime': 61.8709, 'train_samples_per_second': 58.186, 'train_steps_per_second': 2.424, 'total_flos': 117581414400000.0, 'train_loss': 0.09098412831624349, 'epoch': 150.0})

In [ ]:
#import matplotlib.pyplot as plt

#print(trainer.state.log_history)
#print(type(trainer.state.log_history))
#print(len(trainer.state.log_history))
#loss_list = []
#for element in trainer.state.log_history[:100]:

#  loss_list.append(element['loss'])

#x = [i for i in range(0,100)]
#y = loss_list
#plt.plot(x, y)
#plt.show() 

In [ ]:
# Сохраняем модель
model.save_pretrained("drive/MyDrive/Colab Notebooks/ods/model/Mayakovsky_150")


Configuration saved in drive/MyDrive/Colab Notebooks/ods/model/Mayakovsky_150/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/ods/model/Mayakovsky_150/pytorch_model.bin


In [ ]:
# Загружаем сохраненную модель
model2 = GPT2LMHeadModel.from_pretrained("drive/MyDrive/Colab Notebooks/ods/model/Mayakovsky_150").to(DEVICE)

loading configuration file drive/MyDrive/Colab Notebooks/ods/model/Mayakovsky_150/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "use_cache": true,
  "vocab_size": 50264
}

loading weights fi

**Теперь попробуем генерацию текста и посмотрим, получилось ли скопировать стиль Маяковского с короткими, отрывистыми фразами.**
Так как метрики bertscore и BLEURT требуют для вычисления два текста, сгенерированный и тот с которым будем сравнивать, а у меня не задача перевода или перефразирования текста, а что назыается генерация с нуля, то придётся оценивать качество сгенерированного моделью текста, что называется "на глаз".

In [ ]:
max_length=80

In [ ]:
# 1)  Greedy Search он же аргмаксная генерация. Выбирается токен, у которого максимальная вероятность.
# Не лучший метод, так как часто приводит повторению слов. 

text = "В лесу родилась елочка\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model2.eval()
with torch.no_grad():
    out = model2.generate(input_ids, do_sample=False, max_length=max_length)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В лесу родилась елочка
и в дорогу
убежала елочка.
К нам в гости
пришел Новый год.
Он большой, красивый,
и добрый.
Снеговик
убежал, хлопнув дверью.
А у нас
в доме
взорвался бидон
с соленых огурцов.
Все в доме
плесне


In [ ]:
# 2)  Beam search он же поиск по лучу. Здесь на каждом шаге выбирается несколько
# токенов (зависит от ширины луча) и дальше поиск продолжается для каждого из токенов.
# В итоге выбирается вариант с лучшей перплексией.

text = "В лесу родилась елочка\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    out = model2.generate(input_ids, do_sample=True, num_beams=4, max_length=max_length)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В лесу родилась елочка
и в дорогу
убежала елочка.
К нам в гости
пришел Дед Мороз
и сказал:
— Детишки,
каждый день
над крышами
горизонтов
проносятся рой
червей и комаров.
Каждую ночь
над крышами
горизонтов
проносятся


In [ ]:
# 3) Сэмплирование с температурой. Используется для придания тексту "человечности"
# и непредсказуемости. В этом случае берется не самый вероятный токен, а выбор 
# делается "случайно" с учетом распределения вероятности. Параметр температуры
# позволяет контролировать степень случайности. При нулевой температуре получается аргмаксное сэмплирование.
# А при большой наугад. Советуют использовать температуру от 0.8 до 2.

text = "В лесу родилась елочка\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    out = model2.generate(input_ids, do_sample=True,temperature=1.1, max_length=max_length)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В лесу родилась елочка
и во все двери
вежливыми
нотами
сообщают:
это —
плохо
и вполне
даже
плохо
для детей…
Все, что нужно
домашнему ребенку,
достает
мама
из погреба,
а маленький ребенок
достает
дрожалку
и начинает
поить
мою


In [ ]:
# 4) Nucleus Sampling. Сэмплирование с ограничением маловероятных токенов.
# Для запрета сэмплирования маловероятных токенов вводят top-k или top-p ограничения.
# При top-k зануляются все вероятности кроме k самых больших.
# При top-p остается набор токенов, сумма вероятностей которых не больше p. 

text = "В лесу родилась елочка\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    out = model2.generate(input_ids, do_sample=True,temperature=1.2, top_k=25,
                     top_p=0.8, max_length=max_length)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В лесу родилась елочка
и в окошко
звездою
вылетел зайчик.
И все засмеялись
над зайчиком
и назвали его
Зайчиком.

Зайчик
веселится,
улыбаясь,
и говорит:
«Зайка,
я очень маленький,
пойду
к тебе
и сразу спрячусь.



Смысл получившихся текстов конечно оставляет желать лучшего, так как представляет собой шизофрению. Но сети удалось "ухватить" стиль Маяковского с 
короткими, отрывистыми фразами. Возможно с более крупной моделью в тексте было бы больше смысла. 

Статья про BLEURT
https://neurohive.io/ru/novosti/bleurt-metrika-dlya-ocenki-modelej-dlya-generacii-teksta/ 

Метрики, сэплирование.
https://spb.hse.ru/mirror/pubs/share/480745430.pdf   

Бертскор
https://colab.research.google.com/drive/1kpL8Y_AnUUiCxFjhxSrxCsc6-sDMNb_Q  

Бертскор
https://paperswithcode.com/paper/bertscore-evaluating-text-generation-with 

Статья про генерацию текста
https://huggingface.co/blog/how-to-generate

Бертскор
https://github.com/huggingface/datasets/blob/master/metrics/bertscore/bertscore.py

Fine-tune a non-English GPT-2 Model with Huggingface
https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb